In [157]:
import sys
sys.path.append('/home/miketheologitis/MapReduce-Implementation')

from mapreduce.cluster.local_cluster import LocalCluster

In [158]:
local_cluster = LocalCluster(
    n_workers=4,
    n_masters=1,
    initialize=True,
    verbose=False
)

In [159]:
local_monitoring = local_cluster.local_monitoring

In [160]:
import random
import string

def generate_random_string(str_len):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(str_len))

def generate_random_string_list(n, str_len):
    return [generate_random_string(str_len) for _ in range(n)]

# Example usage
data = generate_random_string_list(n=500000, str_len=20)

In [161]:
data[:10]

['fazqhmcmsdpwzsovowzw',
 'wepklddjuvuikpfdoppv',
 'pqsjcvsahgsnayzexdyc',
 'xexsvjifnpincnkkvjeu',
 'dgqamnuljtpakxewizja',
 'inzskkfanzziycrfuuwy',
 'ebicfzavuxjpapyvyxkv',
 'hyebnwafhrhemdmofybl',
 'thpxdfmguckgtvirhhiy',
 'qoxxgrdljncjozdwxlsv']

In [162]:
import sys
print(f"Size: {round(sys.getsizeof(data) / (1024 * 1024), 3)} MB")

Size: 3.974 MB


In [163]:
def map_func(data):
    result = []
    for string in data:
        for char in string:
            result.append((char, 1))
    return result

def reduce_func(data):
    return sum(data)

In [167]:
res1 = local_cluster.mapreduce(data, map_func, reduce_func)

In [171]:
!docker kill ac6dd1b99870

ac6dd1b99870


In [134]:
res1

<Future at 0x7fc50f36f250 state=finished returned list>

In [173]:
local_monitoring.print_zoo()


----------------- Zoo Masters -----------------
Master b6f108b59f4b :  MasterInfo(state='nothing')

----------------- Zoo Workers -----------------
Worker ac6dd1b99870 :  WorkerInfo(state='idle')
Worker fd3fc1fe66b1 :  WorkerInfo(state='idle')
Worker 279a65a92e28 :  WorkerInfo(state='idle')

----------------- Zoo Map Tasks -----------------
Task 0_0 :  Task(state='completed', worker_hostname='ac6dd1b99870')
Task 0_1 :  Task(state='completed', worker_hostname='ac6dd1b99870')
Task 0_2 :  Task(state='completed', worker_hostname='fd3fc1fe66b1')
Task 0_3 :  Task(state='completed', worker_hostname='279a65a92e28')

----------------- Zoo Shuffle Tasks -----------------
Task 0 :  Task(state='completed', worker_hostname='ac6dd1b99870')

----------------- Zoo Reduce Tasks -----------------
Task 0_9_10_11_12_13_14_15_16_17 :  Task(state='completed', worker_hostname='fd3fc1fe66b1')
Task 0_18_19_20_21_22_23_24_25 :  Task(state='completed', worker_hostname='279a65a92e28')
Task 0_0_1_2_3_4_5_6_7_8 : 

In [165]:
local_monitoring.print_hdfs('jobs')

In [174]:
local_cluster.shutdown_cluster(delete_hdfs_jobs=True)